In [1]:
# import packages
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from pandas import read_csv

In [2]:
# read in data

# read csv file


urban_docs = read_csv('/Users/katherinemead/Documents/GitHub/anly-501-project-kam515/501-project-website/codes/01-data-gathering/Urban_API_Endpoints - Sheet1.csv')

# level + _ + source + _ + topic + _ + year + .json

# create a list of all the json files
json_files = []
for i in range(len(urban_docs)):
    json_files.append(str(urban_docs['Level'][i]) + '_' + str(urban_docs['Source'][i]) + '_' + str(urban_docs['Topic'][i]) + '_')

years = ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

new_json_files = []
stem_names = []
for i in json_files:
    for year in years:
        new_json_files.append('/Users/katherinemead/Documents/Urban_Institute_Files/' + i + (year + '.json'))
        stem_names.append(i + year)



error_list = []
working_list = []
working_stem_names = []
working_counter = 0
error_counter = 0
indexer = 0
for i in new_json_files:
    indexer += 1
    try:
        with open(i) as json_file:
            data = json.load(json_file)
            working_counter += 1
            working_list.append(i)
            working_stem_names.append(stem_names[indexer-1])
    except:
        error_list.append(i)
        error_counter += 1



In [3]:
import re

In [4]:
def listify(i):
    with open(i) as file: # Use file to refer to the file object
        # read the file and remove all escape characters
        data = file.read()
        json_string = json.dumps(data, skipkeys=True)  
        list_of_districts = json_string.split("{")
        return list_of_districts


In [5]:
def remove_ugly(stringy):
    s = stringy.replace("\\n", "")
    n_string = s.replace("\\", "")
    p_string = "{" + n_string
    return p_string

def make_pretty_list(listy):
    new_pretty_list = []
    for m in listy:
        new_pretty_list.append(remove_ugly(m))
    return new_pretty_list
        

In [6]:

# pass document name into function
def spit_out_df(document):
    list_of_jsons_for_doc = []
    list_of_strings_that_didnt_jsonify = []
    
    # slashless_long_doc_string = make_pretty_list(listify(document))
    slashless_long_doc_string = make_pretty_list(listify(document))
    
    # go through document and remove commas after } .replace("},", "}") and load into json
    district_counter = 0
    for district_string in slashless_long_doc_string:
        district_counter += 1
        json_ready_string = district_string.replace("},", "}")
        try:
            json_yaas = json.loads(json_ready_string)
            list_of_jsons_for_doc.append(json_yaas)
        except:
            string_to_append = document + " string number: " + str(district_counter)
            list_of_strings_that_didnt_jsonify.append(string_to_append)
    
    # Create dataframe
    items_to_examine = []
    for item in list_of_jsons_for_doc:
        items_to_examine.append(item)
    records = pd.DataFrame.from_records(items_to_examine) 
    return records, list_of_strings_that_didnt_jsonify



In [7]:
# ON DOCUMENT LEVEL

# list_of_strings_that_didnt_jsonify -> loop through all docs and build a master list of all strings that didn't jsonify
def ll_that_didnt_json(l):
    ll_list = []
    for document in l:
        ready_for_master_list = spit_out_df(document)[1]
        ll_list.append(ready_for_master_list)
    return ll_list
    

In [8]:
list_of_docs_that_didnt_csvify = []
doc_num = 0 
list_of_string_level_problems = []
s_df = pd.DataFrame()
for doc in working_list:
    #list_of_string_level_problems.append(ll_that_didnt_json(doc))
    doc_num += 1
    try:
        df = spit_out_df(doc)[0]
        doc_name = '/Users/katherinemead/Documents/Urban_Institute_Files/Csv_files/' + str(working_stem_names[doc_num-1]) + '.csv'
        df.to_csv(doc_name)
        s_problems = list_of_string_level_problems.append(spit_out_df(doc)[1])
        s_df[doc] = s_problems
    except:
        list_of_docs_that_didnt_csvify.append(doc)
s_df.to_csv('/Users/katherinemead/Documents/Urban_Institute_Files/Csv_files/String_Problems.csv')
doc_probs = pd.DataFrame.from_records(list_of_docs_that_didnt_csvify)
doc_probs.to_csv('/Users/katherinemead/Documents/Urban_Institute_Files/Csv_files/Doc_Problems.csv')  